# Spark - Exercício prático

Este notebook contém o exercício prático de Spark usando python. Você pode realizá-lo localmente em sua máquina (se estiver utilizando Linux) ou subir o código no Google Colab. Se estiver usando Windows, verifique o doc [2-2-spark-exercicios](2-2-spark-exercicios.md) para um tutorial de instalação do Spark no Windows.

## Configuração do ambiente. 

As próximos células fazem a instalação e configuração do java-8 e do spark em sua máquina.

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

## 01 - Leitura de dados

Para começar, faça a leitura do arquivo `titanic.csv` para o ambiente spark. Algumas informações importantes:

- O arquivo tem ";" como separador,
- O arquivo possui os nomes das colunas na primeira linha,
- O arquivo deve ser lido com os tipos corretos.

In [2]:
url = "https://raw.githubusercontent.com/neylsoncrepalde/titanic_data_with_semicolon/main/titanic.csv"

# Desenvolva seu código de leitura aqui:
import pyspark.sql.types as t
import pyspark.sql.functions as f
import pandas as pd

path = "base_teste/titanic.csv"

df = (
    spark.read.format("csv")
    .option('inferSchema', 'true')
    .option('header', 'true')
    .option('sep',';')
    .option('encoding','ISO-8859-1')
    .load(path)
)


In [3]:
# Tratando nomes das colunas e valores
df = (
df
    .withColumn("Sexo", (f
                            .when(f.col("Sex") == 'male', "Homem")
                            .when(f.col("Sex") == 'female', "Mulher"))).drop(f.col("Sex"))
    .withColumn("Idade", f.col("Age")).drop(f.col("Age"))
    .withColumn("Nome", f.col("Name")).drop(f.col("Name"))
    .withColumn("Tarifa", f.col("Fare")).drop(f.col("Fare"))

)


## 02 - Algumas análises preliminares

Exiba na tela:

1) O schema da tabela
2) Os 10 primeiros casos
3) Apenas as pessoas que sobreviveram
4) Apenas as pessoas que não sobreviveram e eram do sexo masculino
5) A média de tarifa paga para cada classe
6) A média de tarifa paga para cada classe e sexo
7) Número de sobreviventes por classe e sexo
8) Número de sobreviventes por sexo e categorias de idade (as categorias de idade devem ser construídas como: abaixo de 18 = criança, entre 18 e 40 = jovem adulto, maior de 40 = adulto maduro).

In [4]:
# 1)
# Desenvolva sua resposta aqui:

df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Idade: double (nullable = true)
 |-- Nome: string (nullable = true)
 |-- Tarifa: double (nullable = true)



In [5]:
# 2)
# Desenvolva sua resposta aqui:

df.limit(10).toPandas()

,PassengerId,Survived,Pclass,SibSp,Parch,Ticket,Cabin,Embarked,Sexo,Idade,Nome,Tarifa
0,1,0,3,1,0,A/5 21171,None,S,Homem,22.0,"Braund, Mr. Owen Harris",7.2500
1,2,1,1,1,0,PC 17599,C85,C,Mulher,38.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",71.2833
2,3,1,3,0,0,STON/O2. 3101282,None,S,Mulher,26.0,"Heikkinen, Miss. Laina",7.9250
3,4,1,1,1,0,113803,C123,S,Mulher,35.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",53.1000
4,5,0,3,0,0,373450,None,S,Homem,35.0,"Allen, Mr. William Henry",8.0500
5,6,0,3,0,0,330877,None,Q,Homem,NaN,"Moran, Mr. James",8.4583
6,7,0,1,0,0,17463,E46,S,Homem,54.0,"McCarthy, Mr. Timothy J",51.8625
7,8,0,3,3,1,349909,None,S,Homem,2.0,"Palsson, Master. Gosta Leonard",21.0750
8,9,1,3,0,2,347742,None,S,Mulher,27.0,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",11.1333
9,10,1,2,1,0,237736,None,C,Mulher,14.0,"Nasser, Mrs. Nicholas (Adele Achem)",30.0708


In [6]:
# 3)
# Desenvolva sua resposta aqui:

df3 = (
    df.filter(f.col("Survived") == 1)
    .select("*")  
)
df3.toPandas()

,PassengerId,Survived,Pclass,SibSp,Parch,Ticket,Cabin,Embarked,Sexo,Idade,Nome,Tarifa
0,2,1,1,1,0,PC 17599,C85,C,Mulher,38.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",71.2833
1,3,1,3,0,0,STON/O2. 3101282,None,S,Mulher,26.0,"Heikkinen, Miss. Laina",7.9250
2,4,1,1,1,0,113803,C123,S,Mulher,35.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",53.1000
3,9,1,3,0,2,347742,None,S,Mulher,27.0,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",11.1333
4,10,1,2,1,0,237736,None,C,Mulher,14.0,"Nasser, Mrs. Nicholas (Adele Achem)",30.0708
...,...,...,...,...,...,...,...,...,...,...,...,...
337,876,1,3,0,0,2667,None,C,Mulher,15.0,"""Najib, Miss. Adele Kiamie """"Jane""""""",7.2250
338,880,1,1,0,1,11767,C50,C,Mulher,56.0,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",83.1583
339,881,1,2,0,1,230433,None,S,Mulher,25.0,"Shelley, Mrs. William (Imanita Parrish Hall)",26.0000
340,888,1,1,0,0,112053,B42,S,Mulher,19.0,"Graham, Miss. Margaret Edith",30.0000


In [7]:
# 4)
# Desenvolva sua resposta aqui:

df4 = (
    df.filter((f.col("Survived") == 1) & (f.col("Sexo") == 'Homem'))
    .select("*")  
)
df4.toPandas()

,PassengerId,Survived,Pclass,SibSp,Parch,Ticket,Cabin,Embarked,Sexo,Idade,Nome,Tarifa
0,18,1,2,0,0,244373,None,S,Homem,NaN,"Williams, Mr. Charles Eugene",13.0000
1,22,1,2,0,0,248698,D56,S,Homem,34.0,"Beesley, Mr. Lawrence",13.0000
2,24,1,1,0,0,113788,A6,S,Homem,28.0,"Sloper, Mr. William Thompson",35.5000
3,37,1,3,0,0,2677,None,C,Homem,NaN,"Mamee, Mr. Hanna",7.2292
4,56,1,1,0,0,19947,C52,S,Homem,NaN,"Woolner, Mr. Hugh",35.5000
...,...,...,...,...,...,...,...,...,...,...,...,...
104,839,1,3,0,0,1601,None,S,Homem,32.0,"Chip, Mr. Chang",56.4958
105,840,1,1,0,0,11774,C47,C,Homem,NaN,"Marechal, Mr. Pierre",29.7000
106,858,1,1,0,0,113055,E17,S,Homem,51.0,"Daly, Mr. Peter Denis",26.5500
107,870,1,3,1,1,347742,None,S,Homem,4.0,"Johnson, Master. Harold Theodor",11.1333


In [8]:
# 5)
# Desenvolva sua resposta aqui:

cols = [f.col('Pclass'),f.col('Tarifa')]

df5 = (
    df.select(cols)
    .groupBy(f.col('Pclass').alias("Classe"))
    .agg(f.round(
            f.mean(
                f.col('Tarifa')),2).alias("Média Tarifa Paga"))
    .orderBy(f.col('Pclass'))
)
df5.toPandas()

,Classe,Média Tarifa Paga
0,1,84.15
1,2,20.66
2,3,13.68


In [9]:
# 6)
# Desenvolva sua resposta aqui:

cols = [f.col('Pclass'),f.col('Tarifa'), f.col('Sexo')]

df6 = (
    df.select(cols)
    .groupBy(f.col('Pclass').alias("Classe"),f.col('Sexo'))
    .agg(f.round(
            f.mean(
                f.col('Tarifa')),2).alias("Média Tarifa Paga"))
    .orderBy(f.col('Pclass'))
)
df6.toPandas()

,Classe,Sexo,Média Tarifa Paga
0,1,Mulher,106.13
1,1,Homem,67.23
2,2,Mulher,21.97
3,2,Homem,19.74
4,3,Mulher,16.12
5,3,Homem,12.66


In [10]:
# 7)
# Desenvolva sua resposta aqui:

cols = [f.col('PassengerId'),f.col('Pclass'),f.col('Tarifa'), f.col('Sexo')]

df7 = (
    df.select(cols)
    .groupBy(f.col('Pclass').alias("Classe"),f.col('Sexo').alias("Sexo"))
    .agg(f.count("PassengerId").alias("Quantidade de Sobreviventes"))
    .orderBy(f.col('Pclass'))
)
df7.toPandas()

,Classe,Sexo,Quantidade de Sobreviventes
0,1,Mulher,94
1,1,Homem,122
2,2,Mulher,76
3,2,Homem,108
4,3,Mulher,144
5,3,Homem,347


In [11]:
# 8)
# Desenvolva sua resposta aqui:

# Construindo coluna Categoria Idade
coluna_categoria_idade = (
    f
    .when(f.col("Idade") < 18, "Criança")
    .when((f.col("Idade") >= 18) & (f.col("Idade") <= 40), "Jovem Adulto")
    .when(f.col("Idade") > 40, "Adulto Maduro")
    .when(f.col("Idade").isNull(), "Sem Idade Definida")
)

df8 = df.withColumn("Categoria Idade",coluna_categoria_idade)

cols = [f.col("Categoria Idade"),f.col("Sexo"),f.col("PassengerId")]

df8 = (
    df8.select(cols)
    .groupBy(f.col('Categoria Idade'),f.col('Sexo'))
    .agg(f.count("PassengerId").alias("Quantidade de Sobreviventes"))
    .orderBy(f.col('Categoria Idade'))
)
df8.toPandas()

,Categoria Idade,Sexo,Quantidade de Sobreviventes
0,Adulto Maduro,Homem,102
1,Adulto Maduro,Mulher,48
2,Criança,Mulher,55
3,Criança,Homem,58
4,Jovem Adulto,Homem,293
5,Jovem Adulto,Mulher,158
6,Sem Idade Definida,Homem,124
7,Sem Idade Definida,Mulher,53


## 03 - Escreve dados

Faça uma seleção apenas das pessoas que sobreviveram e escreva os dados em um arquivo **parquet** particionado por sexo e classe.

In [12]:
# Desenvolva a resposta aqui...

df_sobreviventes = (
    df.filter(f.col("Survived") == 1)
    .select("*")
)

(
df_sobreviventes
    .write
    .mode('overwrite')
    .partitionBy('Sexo','Pclass')
    .format('parquet')
    .save('base_teste/parquet')
)